# Create Heatmaps for significant BCR Signaling genes

This notebook looks at the significant genes in at least one cancer. Pancancer heatmaps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [9]:
prot_enr.res2d.iloc[[11]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
11,BCR signaling pathway Homo sapiens acbf44e2-61...,18/64,0.001059,0.018441,0,0,2.154347,14.758576,MAP4K1;MAP2K1;SYK;CHUK;PDPK1;PTEN;PIK3R1;POU2F...,NCI-Nature_2016


# Step 2: Get the list of significant genes 

In [10]:
i = 11
trans = prot_enr.res2d.Genes[i]
genes = trans.split(';')
print(prot_enr.res2d.Term[i])
print('total genes:',len(genes))

BCR signaling pathway Homo sapiens acbf44e2-618c-11e5-8ac5-06603eb7f303
total genes: 18


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [11]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

18

In [14]:
# Only include p-values < a 
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [15]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [16]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [17]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MAPK3', 'SYK', 'PLCG2', 'PIK3R1', 'ELK1', 'MAP4K1'], dtype=object)

In [18]:
# Only include p-values < a
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [19]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=400)

Loading BokehJS ...

# Sig in multiple cancers

In [20]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [22]:
enr2.res2d.iloc[[1]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
1,BCR signaling pathway Homo sapiens acbf44e2-61...,10/64,5.157805e-08,0.000005,0,0,10.048232,168.611028,MAP4K1;PPP3CB;PTPRC;SYK;CHUK;PIK3CA;PTEN;BLNK;...,NCI-Nature_2016


In [24]:
i = 1
trans2 = prot_enr.res2d.Genes[i]
genes_mult = trans2.split(';')
print(prot_enr.res2d.Term[i])
print('total genes:',len(genes_mult))

Class I PI3K signaling events Homo sapiens 12b82bb7-6191-11e5-8ac5-06603eb7f303
total genes: 19


In [25]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['PTEN', 'CYTH1', 'SYK', 'BTK', 'PLCG2', 'PIK3CD', 'BLNK', 'PIK3CA',
       'ZAP70'], dtype=object)

In [26]:
# Only include p-values < a
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [28]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

In [29]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['ADAP1', 'SYK', 'PLCG2', 'PIK3R1'], dtype=object)

In [30]:
# Only include p-values < a
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [31]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=400)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [32]:
enr3 = gp.enrichr(gene_list = genes_mult, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [33]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Class I PI3K signaling events Homo sapiens 12b...,19/48,2.682838e-52,5.607131e-50,0,0,416.666667,49478.145749,HSP90AA1;SYK;PLEKHA1;PDPK1;PTEN;PIK3CD;PIK3R1;...,NCI-Nature_2016
1,BCR signaling pathway Homo sapiens acbf44e2-61...,8/64,5.130589e-16,5.361465e-14,0,0,131.578947,4632.386985,SYK;PIK3CA;PDPK1;PTEN;BLNK;BTK;PLCG2;PIK3R1,NCI-Nature_2016
2,PDGFR-beta signaling pathway Homo sapiens c901...,9/128,1.184280e-15,8.250481e-14,0,0,74.013158,2543.805726,HCK;RAP1A;PIK3CA;LCK;PTEN;PIK3CD;PIK3R1;PLCG1;...,NCI-Nature_2016
3,CXCR4-mediated signaling events Homo sapiens 4...,8/100,2.119525e-14,1.107452e-12,0,0,84.210526,2651.368367,HCK;PIK3CA;PDPK1;LCK;PTEN;PIK3CD;PIK3R1;PIK3CG,NCI-Nature_2016
4,FAS (CD95) signaling pathway Homo sapiens 79cc...,6/38,8.280653e-13,3.461313e-11,0,0,166.204986,4623.770256,SYK;PIK3CA;PDPK1;BTK;PIK3CD;PIK3R1,NCI-Nature_2016


In [23]:
i = 4
print(enr3.res2d.Term[i])
enr3.res2d.Genes[i]

FAS (CD95) signaling pathway Homo sapiens 79cc9c14-6192-11e5-8ac5-06603eb7f303


'SYK'

In [24]:
# Look at p-values 
certain_gene = 'SYK'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
499,SYK,0.000320,-0.697550,Gbm
11632,SYK,0.001919,-0.225323,Hnscc
23793,SYK,0.021461,-0.489000,Luad
36955,SYK,0.155625,-0.475100,Lscc
53051,SYK,0.905056,0.020400,Brca
53874,SYK,0.003901,0.405618,Ov
66600,SYK,0.085643,0.079000,En
81038,SYK,0.775873,0.034700,Colon
